In [51]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.wkt import loads 
from shapely.ops import substring
import matplotlib.pyplot as plt 

In [57]:
case = 'fairfax'
link_df = pd.read_csv('../traffic_inputs/{}/new_{}_links_tmp.csv'.format(case, case))
link_df['fft'] = link_df['length'] / link_df['maxmph'] / 1609 * 3600
link_df = link_df[['link_id', 'link_type', 'geometry','length','fft']]
for t in range(0, 28900, 1000):
    path = './{}/with_bg/agent_stats_add_srsn_reroute_rtt/{}_vect_at_{}_bg_1_real_departure_time_agent.csv'.format(case,case,t)
    agents_df = pd.read_csv(path)
    agents_df = agents_df[~agents_df['current_link'].isnull()]
    agents_df['current_link'] = agents_df['current_link'].astype('str')
    agents_df['initial_letter_current_link'] = agents_df['current_link'].apply(lambda x: x[0])
    agents_df = agents_df[agents_df['initial_letter_current_link'] != 'v']
    agents_df['current_link'] = agents_df['current_link'].astype('float')
    agents_df['current_link'] = agents_df['current_link'].astype('int')    
    agents_df.reset_index(drop = True, inplace = True) 
    link_travel_time_dict = ((t * agents_df.loc[agents_df['agent_status'].isin([2])].groupby('current_link').size() - 
                                    agents_df.loc[agents_df['agent_status'].isin([2])].groupby('current_link')['current_link_enter_time'].sum())/
                                    agents_df.loc[agents_df['agent_status'].isin([2])].groupby('current_link').size()).to_dict()

    link_df[t] = link_df['link_id'].map(link_travel_time_dict).fillna(link_df['fft'])
    link_df['average_speed_{}'.format(t)] = link_df['length'] / link_df[t]

In [61]:
link_df = link_df[link_df['link_type'] != 'vl_link']

2        11.173611
3        11.173611
4        11.173611
5        11.173611
6        11.173611
           ...    
40205    13.408333
40206    19.393829
40207    13.408333
40208    29.051389
40209    13.408333
Name: average_speed_1000, Length: 40208, dtype: float64

In [82]:
case = 'fairfax'
basemap = pd.read_csv('../traffic_inputs/fairfax/new_fairfax_links_tmp.csv')
basemap = gpd.GeoDataFrame(basemap, crs='epsg:4326', geometry=basemap['geometry'].map(loads)).to_crs(3857)

for t in range(0, 28900, 1000):
    link_gdf = gpd.GeoDataFrame(link_df, crs='epsg:4326', geometry=link_df['geometry'].map(loads)).to_crs(3857)
    fig, ax = plt.subplots(figsize=(10, 10))
    basemap.plot(ax=ax, lw=0.2, color='gray')
    link_gdf.plot(column = 'average_speed_{}'.format(t), ax = ax, lw =1, cmap='RdYlGn', legend = True, legend_kwds
                  ={'label': "speed [m/s]"})
    ax.set_xlim([-13652000,-13630802]) #fairfax
    ax.set_ylim([4560000,4600611]) #fairfax
    ax.axis('off')
    ax.text(0, 0.1, 't{}'.format(t), fontsize = 18, transform=ax.transAxes)
    plt.savefig('images/{}/speed/speed_t{}.png'.format(case,t),dpi= 200)
    plt.close()

In [39]:
link_df['count_num'] = 0
for i, row in link_df.iterrows():
    for t in range(0, 28900, 1000):
        if row[t] >= 4 * row['fft']:
            link_df.loc[i, 'count_num'] += 1

link_df.to_csv('./{}_tmp.csv'.format(case), index = False)

link_df_1 = link_df[link_df['count_num']>=8].copy()
link_df_1.reset_index(drop = True, inplace = True)
link_df_1 = link_df_1[['link_id','geometry','count_num']]
link_df_1.to_csv('./{}_bottleneck_selected.csv'.format(case), index = False)